In [1]:
import pymongo
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
connection_string = "mongodb://localhost:27017"
client = pymongo.MongoClient(connection_string)
db = client['MemoireEsgf']
collection = db['PYPL']
df2 = pd.DataFrame(list(collection.find()))
data2 = df2.drop('_id', axis=1)
data2.tail()

,Date,Open,High,Low,Close,Volume
269,2023-08-25,60.990002,61.599998,60.080002,61.189999,11134000
270,2023-08-28,61.590000,61.959999,61.029999,61.669998,10459300
271,2023-08-29,61.630001,62.740002,61.529999,62.500000,11627600
272,2023-08-30,62.380001,63.470001,62.119999,63.419998,12041000
273,2023-08-31,62.720001,63.410000,62.450001,62.509998,13486397


In [3]:
data_name2 = 'MemoireIndiceMarket'
db2 = client[data_name2]
collection2 = db2.ASX_200

In [4]:
df2 = pd.DataFrame(list(collection2.find()))
data2 = df2.drop('_id', axis=1)
data2.tail()

,Date,Open,High,Low,Close,Volume
2812,2023-08-15,7272.700195,7329.299805,7271.899902,7305.000000,610600
2813,2023-08-16,7284.500000,7284.500000,7185.200195,7195.200195,714000
2814,2023-08-17,7190.799805,7191.299805,7106.799805,7146.000000,897900
2815,2023-08-18,7151.500000,7167.700195,7124.500000,7148.100098,806300
2816,2023-08-21,7145.799805,7145.899902,7115.500000,7115.500000,711900


In [9]:
#pip install nalyst

In [8]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import warnings
#import nalyst
warnings.filterwarnings("ignore")

In [ ]:
class MonteCarloSimulator:
    def __init__(self, symbol, start_date, end_date):
        self.symbol = symbol
        self.data = yf.download(symbol, start=start_date, end=end_date)['Adj Close']
        self.returns = np.log(1 + self.data.pct_change())
        self.mu = self.returns.mean()
        self.sigma = self.returns.std()

    def generate_paths(self, num_paths, num_days):
        T = num_days
        N = num_paths
        S0 = self.data.iloc[-1]
        dt = 1 / T

        paths = np.zeros((T, N))
        paths[0] = S0
        for t in range(1, T):
        drift = (self.mu - 0.5 * self.sigma ** 2) * dt
        diffusion = self.sigma * np.sqrt(dt) * np.random.normal(size=N)
        paths[t] = paths[t-1] * np.exp(drift + diffusion)

        return paths

    def plot_paths(self, num_paths, num_days):
        paths = self.generate_paths(num_paths, num_days)
        plt.style.use('dark_background')
        plt.figure(figsize=(15,6))
        plt.plot(paths)
        plt.xlabel('Trading Days')
        plt.ylabel('Stock Price')
        plt.title('Monte Carlo Simulation for ' + self.symbol)
        plt.grid(linestyle='--', color='gray', alpha=0.5)
        plt.show()

In [ ]:
# (Ractif − Rf) / Ecart-type(Ractif)
class SharpRatio:
    def __init__(self, ticker):
        self.ticker = yf.Ticker(ticker)
        self.returns = self.ticker.history(period="max")['Close'].pct_change().dropna()

    def calculate_sharp_ratio(self, cumulative_returns):
        average_returns = np.mean(cumulative_returns, axis=1)
        risk_free_rate = 0.031690
        sharp_ratios = (average_returns - risk_free_rate) / np.std(average_returns)
        return sharp_ratios

    def plot_sharp_ratio(self, sharp_ratios, title):
        plt.figure(figsize=(16, 9))
        plt.grid(linestyle='--', color='gray', alpha=0.5)
        plt.hist(sharp_ratios, bins=50, color='red', alpha=0.7)
        plt.title(title)
        plt.xlabel('Sharp Ratio')
        plt.show()

In [10]:
# the benchmark return as the daily return of the index
benchmark_return = data2[['Date','Close']]
benchmark_return['Rentabilite'] = benchmark_return['Close'].pct_change().dropna()
benchmark_return

,Date,Close,Rentabilite
0,2012-07-02,4133.000000,NaN
1,2012-07-03,4127.200195,-0.001403
2,2012-07-04,4172.200195,0.010903
3,2012-07-05,4169.200195,-0.000719
4,2012-07-06,4157.799805,-0.002734
...,...,...,...
2812,2023-08-15,7305.000000,0.003848
2813,2023-08-16,7195.200195,-0.015031
2814,2023-08-17,7146.000000,-0.006838
2815,2023-08-18,7148.100098,0.000294


# BETA

In [6]:
import pandas as pd
d = {'l': ["b","R","L",'O',"p"],
      'm': ["b","x","L",'O',"v","p"]}


df = pd.DataFrame(data=d)
df

ValueError: All arrays must be of the same length

In [4]:
k = {'ff':0,
     'g':8}

k['ff'] = 9
k


{'ff': 9, 'g': 8}

In [ ]:
def calculate_beta_max(ticker, benchmark_ticker):
    stock = yf.Ticker(ticker).history(period="max")
    benchmark = yf.Ticker(benchmark_ticker).history(period="max")

    stock_returns = stock['Close'].pct_change().dropna()
    benchmark_returns = benchmark['Close'].pct_change().dropna()

    min_len = min(len(stock_returns), len(benchmark_returns))
    stock_returns = stock_returns[-min_len:]
    benchmark_returns = benchmark_returns[-min_len:]

    covariance = np.cov(stock_returns, benchmark_returns)[0][1]
    benchmark_variance = np.var(benchmark_returns)
    beta = covariance / benchmark_variance

    return beta

In [ ]:
def calculate_beta(ticker, benchmark):
    stock = yf.Ticker(ticker).history(period="5y", interval='1mo')
    benchmark = yf.Ticker(benchmark).history(period="5y", interval='1mo')

    stock_returns = stock['Close'].pct_change().dropna()
    benchmark_returns = benchmark['Close'].pct_change().dropna()

    covariance = np.cov(stock_returns, benchmark_returns)[0][1]
    benchmark_variance = np.var(benchmark_returns)

    beta = covariance / benchmark_variance
    return beta

In [1]:
import numpy as np
x = [-2.1, -1,  4.3,4]
y = [3,  1.1,  0.12]
#X = np.stack((x, y), axis=0)
#X

In [2]:
min_len = min(len(x), len(y))
stock_returns = x[-min_len:]
benchmark_returns = y[-min_len:]
min_len

3

In [7]:
l = [8,7,6,'y']
for i in l:
    if i == l[-1]:
        print('dernier element')

dernier element


In [8]:
l[-1]

'y'

In [3]:
stock_returns

[-1, 4.3, 4]

In [4]:
benchmark_returns

[3, 1.1, 0.12]

In [7]:
np.cov(x, y)

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 4 and the array at index 1 has size 3

In [9]:
# Chaîne de caractères d'origine
chaine = "pred_Euro_Stoxx_50.csv"

# Divisez la chaîne en fonction du caractère '_'
elements = chaine.split('_')

# Extrayez le deuxième élément (index 1) de la liste résultante
euro_stoxx_50 = elements[1]

print(euro_stoxx_50)

Euro


In [11]:
chaine = "pred_CAC_40.csv"
elements = chaine.split('_')
euro_stoxx_50 = '_'.join(elements[1:])

print(euro_stoxx_50)

CAC_40.csv


In [27]:
# Chaîne de caractères d'origine
fichier = 'Beta_Company_Euro_Stoxx_50.csv'

# Divisez la chaîne en fonction du caractère '_'
elements = fichier.split('_')
nom_fichier = '_'.join(elements[2:])

print(nom_fichier)

['Beta', 'Company', 'Euro', 'Stoxx', '50.csv']
Euro_Stoxx_50.csv
